FLOW
1. データを取得
2. データの整形
　　Q学習のためには離散値にする必要あり(GridWorld)
  　有効数字の桁数等は工夫の余地あり
   入力変数を10個とすると，1つあたり100分割してその組み合わせ
   100^10通りのテーブルが生成されるが，これは現実的なサイズであるのかは不明(100^10=10^20)
   np.digitize(値, 範囲)を用いることで分割された値への変換が容易に可能
3. Q関数の定義
　　強化学習は時間割引(早く行動しないと得られる報酬が減る)を導入するが，それは不必要に思える
   agentの行動によって環境が変化するわけでないことが原因

テーブルを作り，そのテーブルの値で売買を決定すれば良いだけでは？
有意に偏りのあるセルの時のみ売買を行う仕組み
データからGridWorldを形成し，中の値をHighなら+1, Lowなら-1して有意に偏りのあるセルの場合のみ売買
売買しないという選択肢を設けることが効果的では？
試してみる価値あり
  
  
functions
def getdata()
def grider(df)
def maketable()

In [18]:
# -*- coding: utf-8 -*-
from oandapyV20 import API
from oandapyV20.exceptions import V20Error
from oandapyV20.endpoints.pricing import PricingStream
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.instruments as instruments

import shutil
import os

import json
import datetime
import pandas as pd
import numpy as np
import talib as ta
import math


#自分のアカウント、トークンをセット
accountID = "101-009-16415310-001"
access_token = '860c2d2dbbcd01b6d95939b6af6b5981-8664724ce0870220c55161d704137df6'

api = API(access_token=access_token, environment="practice")
        
# Oandaからcandleデータを取得する。
def getCandleDataFromOanda(instrument, api, date_from, date_to, granularity):
    params = {
        "from": date_from.isoformat(),
        "to": date_to.isoformat(),
        "granularity": granularity,
    }
    r = instruments.InstrumentsCandles(instrument=instrument, params=params)
    return api.request(r)

def oandaJsonToPythonList(JSONRes):
    
    data = []
    for res in JSONRes['candles']:
        temp = [((datetime.datetime.fromisoformat(res['time'][:19]))+datetime.timedelta(days=+9)).strftime('%Y-%m-%d %H:%M')]#indexとなる時間を追加
        temp.extend([res['volume'],
            res['mid']['o'],
            res['mid']['h'],
            res['mid']['l'],
            res['mid']['c'],
            ])
        data.append(temp)
    return data

def logarism(x):
    x = abs(x)
    return math.log(x)


def get_now_data(time, period_days, money, foot, back_days = 0, times = 0):
    
    all_data = []
    #time = time.replace(day=7, day=0, second=0, microsecond=0)
    #入力された日の当日の7時
    
    NY = datetime.timedelta(hours=-14)
    back = datetime.timedelta(days=-back_days)
    
    date_to = time + NY + datetime.timedelta(seconds=-1) + back

    date_from = date_to + datetime.timedelta(days=-period_days)
    for i in range(times+1):    
        ret = getCandleDataFromOanda(money, api, date_from, date_to, foot)
        month_data = oandaJsonToPythonList(ret)#取得したデータを格納

        all_data.extend(month_data)#データを追加
        
        date_to = date_from
        date_from = date_to + datetime.timedelta(days=-period_days)

    # pandas DataFrameへ変換
    df = pd.DataFrame(all_data)

#     print(date_from, date_to)
#     print(df)
    
    df.columns = ['Datetime','Volume', 'Open', 'High', 'Low', 'Close']
    
    df['Volume'] = df['Volume'].astype('double')
    df['Open'] = df['Open'].astype('double')
    df['Close'] = df['Close'].astype('double')
    df['High'] = df['High'].astype('double')
    df['Low'] = df['Low'].astype('double')
    
    
    close = np.array(df["Close"]).astype(float)
#     #以下、talibを用いてテクニカル指標（今回の学習で用いる特徴量）を算出しdf_feature入れる
#     #単純移動平均は、単純移動平均値とその日の終値の比を特徴量として用いる
    df["SMA2"]= ta.SMA(close, timeperiod=2) / close
#     df["SMA3"]= ta.SMA(close, timeperiod=3) / close
#     df["SMA4"]= ta.SMA(close, timeperiod=4) / close
    df["SMA5"]= ta.SMA(close, timeperiod=5) / close
#     df["SMA6"]= ta.SMA(close, timeperiod=6) / close
#     df["SMA7"]= ta.SMA(close, timeperiod=7) / close
#     df["SMA8"]= ta.SMA(close, timeperiod=8) / close
#     df["SMA9"]= ta.SMA(close, timeperiod=9) / close
    df["SMA10"]= ta.SMA(close, timeperiod=10) / close
    df["RSI"] = ta.RSI(close, timeperiod=9) / close
    #ボリンジャーバンド Main 
    upper, middle, lower = ta.BBANDS(close, timeperiod=10, nbdevup=2, nbdevdn=2)
    df["BBANDS+σ"] = upper / close * 1000
    df["BBANDS-σ"] = lower / close * 1000
    
    #df['Rate']=(df['Close']-df['Open'])/df['Open']
    
#     df['Open'] = df['Open'].apply(lambda x: logarism(x))
#     df['Close'] = df['Close'].apply(lambda x: logarism(x))
    
    df["SMA2"]= df["SMA2"].apply(lambda x: logarism(x)) * 1000
#     df["SMA3"]= df["SMA3"].apply(lambda x: logarism(x)) * 1000
#     df["SMA4"]= df["SMA4"].apply(lambda x: logarism(x)) * 1000
    df["SMA5"]= df["SMA5"].apply(lambda x: logarism(x)) * 1000
#     df["SMA6"]= df["SMA6"].apply(lambda x: logarism(x)) * 1000
#     df["SMA7"]= df["SMA7"].apply(lambda x: logarism(x)) * 1000
#     df["SMA8"]= df["SMA8"].apply(lambda x: logarism(x)) * 1000
#     df["SMA9"]= df["SMA9"].apply(lambda x: logarism(x)) * 1000
    df["SMA10"]= df["SMA10"].apply(lambda x: logarism(x)) * 1000
    
    
#     df['Rate']=(df['Close'].astype('double')-df['Open'].astype('double'))/df['Open'].astype('double')
    
#     df['Rate'] = df['Rate']*1000
#     df['true']=(df['Close'].astype('double')-df['Open'].astype('double'))/df['Open'].astype('double')
    
    df['class'] = ((df['Close'].astype('double')-df['Open'].astype('double'))/df['Open'].astype('double')).apply(lambda x: classify_two(x))
    df['class'] = df['class'].shift(-1)
    
    df = df.dropna()
    
#     df['Open'] = df['Open']/df['Close']
    
    df = df.drop('Open', axis=1)
    df = df.drop('High', axis=1)
    df = df.drop('Low', axis=1)
    df = df.drop('Close', axis=1)
    df = df.drop('Volume', axis=1)
    
    df = df.set_index('Datetime')
    
    
    return df



def get_csv(back_days, days, money):
    
    time = datetime.datetime.now() 
    
    df = get_now_data(time, period_days=days, money=money, foot = FOOT , back_days=back_days, times=0)
    date_to = (time+datetime.timedelta(days=-back_days)+datetime.timedelta(seconds=-1)).strftime("%Y-%m-%d")
    date_from = (time + datetime.timedelta(days=-days)+datetime.timedelta(days=-back_days)).strftime("%Y-%m-%d")
    df.to_csv("csv/"+str(money)+"_"+ date_from + "_" + date_to +"_"+str(days)+'.csv', mode = 'w')
    print('"'+str(money)+"_"+ date_from + "_" + date_to +"_"+str(days)+'.csv', end='", ')
    
    return df

def get_moneys_csv(back_days, days, moneylist):
    csvlist = []
    
    time = datetime.datetime.now()
    date_to = (time+datetime.timedelta(days=-back_days)+datetime.timedelta(seconds=-1)).strftime("%Y-%m-%d")
    date_from = (time + datetime.timedelta(days=-days)+datetime.timedelta(days=-back_days)).strftime("%Y-%m-%d")
    
    for i in range (len(moneylist)):
        get_csv(back_days, days, money=moneylist[i])
        csvlist.extend([str(moneylist[i])+"_"+ date_from + "_" + date_to +"_"+str(days)+'.csv'])
    
    return csvlist
        
def classify_two(x):
        if x<0:
            return -1
        elif 0<=x:
            return 1
        
def classify_random(x):
        ram = random.randint(0,1)
        if ram==0:
            return -1
        elif ram==1:
            return 1
        

def get_int_csv():
    shutil.rmtree('csv')
    os.mkdir('csv')
    moneylist = ['USD_JPY']
    array = get_moneys_csv(back_days=10, days=4000, moneylist = moneylist)
    return array

def make_bins(max_num, min_num, bin_num):
    global DIVIDE
    delta = max_num-min_num
    delta = delta/(DIVIDE-1)
    bins = []
    povit = min_num
    for i in range(bin_num-1):
        povit += delta
        bins.extend([povit])
    return bins

def digitize_data(df):
    bin_num = len(df)
    for i in range(len(df.columns)):
        col = df.columns.values.tolist()[i]
        if col == "class":
            continue
        elif col == "true":
            continue
        else:
            max_num = df[col].max()
            min_num = df[col].min()
            df[col] = np.digitize(df[col], make_bins(max_num, min_num, bin_num))
    return df

def grider(dataname):
    df = pd.read_csv("csv/"+dataname, index_col='Datetime')
    df = digitize_data(df)
#     print(df)
    return df
    
def main():
    global DIVIDE
    count = 0
    array = get_int_csv()
    df = grider(array[0])
    table = np.zeros((DIVIDE,DIVIDE,DIVIDE,DIVIDE,DIVIDE,DIVIDE))
    for y in range(len(df)):
        temp = []
        for x in range(len(df.columns)-1):
            temp.extend([df.iat[y,x]])
        a = temp[0]
        b = temp[1]
        c = temp[2]
        d = temp[3]
        e = temp[4]
        f = temp[5]
        table[a][b][c][d][e][f] += int(df.iat[y,6])
        count += 1
        
#     print(table)
    print(np.count_nonzero(table>1))
    print(np.count_nonzero(table>1)/np.count_nonzero(table!=0))
    print(count)

FOOT = 'D'
DIVIDE =15
main()

"USD_JPY_2010-01-29_2021-01-11_4000.csv", 124
0.11843361986628462
3127


In [55]:
# -*- coding: utf-8 -*-
from oandapyV20 import API
from oandapyV20.exceptions import V20Error
from oandapyV20.endpoints.pricing import PricingStream
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.instruments as instruments

import shutil
import os

import json
import datetime
import pandas as pd
import numpy as np
import talib as ta
import math


#自分のアカウント、トークンをセット
accountID = "101-009-16415310-001"
access_token = '860c2d2dbbcd01b6d95939b6af6b5981-8664724ce0870220c55161d704137df6'

api = API(access_token=access_token, environment="practice")
        
# Oandaからcandleデータを取得する。
def getCandleDataFromOanda(instrument, api, date_from, date_to, granularity):
    params = {
        "from": date_from.isoformat(),
        "to": date_to.isoformat(),
        "granularity": granularity,
    }
    r = instruments.InstrumentsCandles(instrument=instrument, params=params)
    return api.request(r)

def oandaJsonToPythonList(JSONRes):
    
    data = []
    for res in JSONRes['candles']:
        temp = [((datetime.datetime.fromisoformat(res['time'][:19]))+datetime.timedelta(hours=+9)).strftime('%Y-%m-%d %H:%M')]#indexとなる時間を追加
        temp.extend([res['volume'],
            res['mid']['o'],
            res['mid']['h'],
            res['mid']['l'],
            res['mid']['c'],
            ])
        data.append(temp)
    return data

def logarism(x):
    x = abs(x)
    return math.log(x)


def get_now_data(time, period_hours, money, foot, back_hours = 0, times = 0):
    
    all_data = []
    #time = time.replace(hour=7, hour=0, second=0, microsecond=0)
    #入力された日の当日の7時
    
    NY = datetime.timedelta(hours=-14)
    back = datetime.timedelta(hours=-back_hours)
    
    date_to = time + NY + datetime.timedelta(seconds=-1) + back

    date_from = date_to + datetime.timedelta(hours=-period_hours)
    for i in range(times+1):    
        ret = getCandleDataFromOanda(money, api, date_from, date_to, foot)
        month_data = oandaJsonToPythonList(ret)#取得したデータを格納

        all_data.extend(month_data)#データを追加
        
        date_to = date_from
        date_from = date_to + datetime.timedelta(hours=-period_hours)

    # pandas DataFrameへ変換
    df = pd.DataFrame(all_data)

#     print(date_from, date_to)
#     print(df)
    
    df.columns = ['Datetime','Volume', 'Open', 'High', 'Low', 'Close']
    
    df['Volume'] = df['Volume'].astype('double')
    df['Open'] = df['Open'].astype('double')
    df['Close'] = df['Close'].astype('double')
    df['High'] = df['High'].astype('double')
    df['Low'] = df['Low'].astype('double')
    
    
    close = np.array(df["Close"]).astype(float)
#     #以下、talibを用いてテクニカル指標（今回の学習で用いる特徴量）を算出しdf_feature入れる
#     #単純移動平均は、単純移動平均値とその日の終値の比を特徴量として用いる
    df["SMA2"]= ta.SMA(close, timeperiod=2) / close
#     df["SMA3"]= ta.SMA(close, timeperiod=3) / close
#     df["SMA4"]= ta.SMA(close, timeperiod=4) / close
    df["SMA5"]= ta.SMA(close, timeperiod=5) / close
#     df["SMA6"]= ta.SMA(close, timeperiod=6) / close
#     df["SMA7"]= ta.SMA(close, timeperiod=7) / close
#     df["SMA8"]= ta.SMA(close, timeperiod=8) / close
#     df["SMA9"]= ta.SMA(close, timeperiod=9) / close
    df["SMA10"]= ta.SMA(close, timeperiod=10) / close
    df["RSI"] = ta.RSI(close, timeperiod=9) / close
    #ボリンジャーバンド Main 
    upper, middle, lower = ta.BBANDS(close, timeperiod=10, nbdevup=2, nbdevdn=2)
    df["BBANDS+σ"] = upper / close * 1000
    df["BBANDS-σ"] = lower / close * 1000
    
    #df['Rate']=(df['Close']-df['Open'])/df['Open']
    
#     df['Open'] = df['Open'].apply(lambda x: logarism(x))
#     df['Close'] = df['Close'].apply(lambda x: logarism(x))
    
    df["SMA2"]= df["SMA2"].apply(lambda x: logarism(x)) * 1000
#     df["SMA3"]= df["SMA3"].apply(lambda x: logarism(x)) * 1000
#     df["SMA4"]= df["SMA4"].apply(lambda x: logarism(x)) * 1000
    df["SMA5"]= df["SMA5"].apply(lambda x: logarism(x)) * 1000
#     df["SMA6"]= df["SMA6"].apply(lambda x: logarism(x)) * 1000
#     df["SMA7"]= df["SMA7"].apply(lambda x: logarism(x)) * 1000
#     df["SMA8"]= df["SMA8"].apply(lambda x: logarism(x)) * 1000
#     df["SMA9"]= df["SMA9"].apply(lambda x: logarism(x)) * 1000
    df["SMA10"]= df["SMA10"].apply(lambda x: logarism(x)) * 1000
    
    
#     df['Rate']=(df['Close'].astype('double')-df['Open'].astype('double'))/df['Open'].astype('double')
    
#     df['Rate'] = df['Rate']*1000
#     df['true']=(df['Close'].astype('double')-df['Open'].astype('double'))/df['Open'].astype('double')
    
    df['class'] = ((df['Close'].astype('double')-df['Open'].astype('double'))/df['Open'].astype('double')).apply(lambda x: classify_two(x))
    df['class'] = df['class'].shift(-1)
    
    df = df.dropna()
    
#     df['Open'] = df['Open']/df['Close']
    
    df = df.drop('Open', axis=1)
    df = df.drop('High', axis=1)
    df = df.drop('Low', axis=1)
    df = df.drop('Close', axis=1)
    df = df.drop('Volume', axis=1)
    
    df = df.set_index('Datetime')
    
    
    return df



def get_csv(back_hours, hours, money):
    
    time = datetime.datetime.now() 
    
    df = get_now_data(time, period_hours=hours, money=money, foot = FOOT , back_hours=back_hours, times=0)
    date_to = (time+datetime.timedelta(hours=-back_hours)+datetime.timedelta(seconds=-1)).strftime("%Y-%m-%d")
    date_from = (time + datetime.timedelta(hours=-hours)+datetime.timedelta(hours=-back_hours)).strftime("%Y-%m-%d")
    df.to_csv("csv/"+str(money)+"_"+ date_from + "_" + date_to +"_"+str(hours)+'.csv', mode = 'w')
    print('"'+str(money)+"_"+ date_from + "_" + date_to +"_"+str(hours)+'.csv', end='", ')
    
    return df

def get_moneys_csv(back_hours, hours, moneylist):
    csvlist = []
    
    time = datetime.datetime.now()
    date_to = (time+datetime.timedelta(hours=-back_hours)+datetime.timedelta(seconds=-1)).strftime("%Y-%m-%d")
    date_from = (time + datetime.timedelta(hours=-hours)+datetime.timedelta(hours=-back_hours)).strftime("%Y-%m-%d")
    
    for i in range (len(moneylist)):
        get_csv(back_hours, hours, money=moneylist[i])
        csvlist.extend([str(moneylist[i])+"_"+ date_from + "_" + date_to +"_"+str(hours)+'.csv'])
    
    return csvlist
        
def classify_two(x):
        if x<0:
            return -1
        elif 0<=x:
            return 1
        
def classify_random(x):
        ram = random.randint(0,1)
        if ram==0:
            return -1
        elif ram==1:
            return 1
        

def get_int_csv():
    global BACK
    shutil.rmtree('csv')
    os.mkdir('csv')
    moneylist = ['USD_JPY']
    array = get_moneys_csv(back_hours=BACK, hours=4000, moneylist = moneylist)
    return array

def make_bins(max_num, min_num, bin_num):
    global DIVIDE
    delta = max_num-min_num
    delta = delta/(DIVIDE-1)
    bins = []
    povit = min_num
    for i in range(bin_num-1):
        povit += delta
        bins.extend([povit])
    return bins

def digitize_data(df):
    bin_num = len(df)
    for i in range(len(df.columns)):
        col = df.columns.values.tolist()[i]
        if col == "class":
            continue
        elif col == "true":
            continue
        else:
            max_num = df[col].max()
            min_num = df[col].min()
            df[col] = np.digitize(df[col], make_bins(max_num, min_num, bin_num))
    return df

def grider(dataname):
    df = pd.read_csv("csv/"+dataname, index_col='Datetime')
    df = digitize_data(df)
#     print(df)
    return df
    
def make_table(line, array):
    global DIVIDE
    count = 0
    df = grider(array[0])
    print("Table data")
    print(df.head(1).index.values, end=' to ')
    print(df.tail(1).index.values)
    table = np.zeros((DIVIDE,DIVIDE,DIVIDE,DIVIDE,DIVIDE,DIVIDE))
    for y in range(len(df)):
        temp = []
        for x in range(len(df.columns)-1):
            temp.extend([df.iat[y,x]])
        table[temp[0]][temp[1]][temp[2]][temp[3]][temp[4]][temp[5]] += int(df.iat[y,6])
    print(np.count_nonzero(table>line)/np.count_nonzero(table!=0))
    return table

def buy_test(table, money):
    global FOOT, BACK, LINE, POSSESTION, PAY, PAYOUT
    df_now = get_now_data(datetime.datetime.now(), BACK, money, FOOT, back_hours = 0, times = 0)
    df_now['class'] = df_now['class'].shift(1)
    df_now = digitize_data(df_now)
    df_now = df_now.dropna()
    count = [0,0,0]
    for y in range(len(df_now)):
        temp = []
        for x in range(len(df_now.columns)-1):
            temp.extend([df_now.iat[y,x]])
        value = table[temp[0]][temp[1]][temp[2]][temp[3]][temp[4]][temp[5]]
        if value>=LINE: #HIGH予想
            count[0] += 1
            POSSESTION -= PAY
            if(int(df_now.iat[y,6])>0): #実際HIGHなら
                POSSESTION += PAY*PAYOUT
            else:
                continue
        elif value<=-LINE: #LOW予想
            count[1] += 1
            POSSESTION -= PAY
            if(int(df_now.iat[y,6])<0): #実際LOWなら
                POSSESTION += PAY*PAYOUT
            else:
                continue
        else:
            count[2] += 1
    print(count)
    

def main():
    global LINE
    array = get_int_csv()
    table = make_table(LINE, array)
    buy_test(table, 'USD_JPY')
    print(POSSESTION)
    

FOOT = 'H1'
DIVIDE =5
LINE = 1
BACK = 100
PAY = 1000
PAYOUT = 1.95
POSSESTION = 10000
main()

"USD_JPY_2020-08-03_2021-01-16_4000.csv", 0.22950819672131148
[13, 16, 26]
25850.0


In [54]:
DAYはうまくいかない　データのばらつきが原因？

"USD_JPY_2009-10-31_2020-10-13_4000.csv", 0.225
[20, 13, 28]
6250.0
